catboost version : 1.2.2

xgboost version : 2.0.2

lightgbm version : 4.1.0

optuna version : 3.5.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DACON_daegu

/content/drive/.shortcut-targets-by-id/186PExvilnSbmIxbZAyf1D41wAnaMiiFv/DACON_daegu


In [ ]:
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

# **대구 교통 사고 피해 예측 AI 경진대회**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [ ]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv, countrywide_accident.csv 파일을 로드하여 상위행을 출력해 봅시다

In [ ]:
import pandas as pd

train_org = pd.read_csv('./open/train.csv')
test_org = pd.read_csv('./open/test.csv')
total_org = pd.read_csv('./open/external_open/countrywide_accident.csv')

display(train_org.head())
display(test_org.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


In [ ]:
train_org.shape,test_org.shape

((39609, 23), (10963, 8))

## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [ ]:
display(f"train : {train_org.iloc[0]['사고일시']} ~ {train_org.iloc[-1]['사고일시']}")
display(f"test : {test_org.iloc[0]['사고일시']} ~ {test_org.iloc[-1]['사고일시']}")

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

## **예측 대상 확인하기**

sample_submission.csv를 로드하여 출력 형식을 확인해 봄으로서,
대회 페이지 규칙에 기술되어 있는, 예측 대상이 무엇인지 확인해 봅시다.

최종적으로 예측해야 하는 값은 'ECLO'(인명피해 심각도) 값입니다.

그러나, train데이터 컬럼에 있는 '사망자수', '중상자수', '경상자수', '부상자수'를 각각 예측하여 ECLO 예측값을 구할 수도 있습니다. 즉, ECLO 값을 예측하는 단일 모델로 접근할 수도,  각각을 별도로 예측하는 모델로 접근할 수도 있습니다.

<img src="https://drive.google.com/uc?id=19PvSqu8k04D3a-Bskcl-d-Xkckkn2Fvv">  

In [ ]:
sample_submission = pd.read_csv("./open/sample_submission.csv")
sample_submission.head()

,ID,ECLO
0,ACCIDENT_39609,0
1,ACCIDENT_39610,0
2,ACCIDENT_39611,0
3,ACCIDENT_39612,0
4,ACCIDENT_39613,0


## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다


### **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [ ]:
train_df = train_org.copy()
test_df = test_org.copy()
total_df = total_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

total_df[['연', '월', '일', '시간']] = total_org['사고일시'].str.extract(time_pattern)
total_df[['연', '월', '일', '시간']] = total_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
total_df = total_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

### **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [ ]:
train_org['시군구'].value_counts()

대구광역시 남구 대명동      1749
대구광역시 수성구 범어동     1173
대구광역시 달서구 상인동     1143
대구광역시 서구 비산동       968
대구광역시 서구 내당동       938
                  ... 
대구광역시 중구 도원동         2
대구광역시 동구 신용동         2
대구광역시 중구 동성로1가       2
대구광역시 북구 도남동         2
대구광역시 동구 평광동         1
Name: 시군구, Length: 199, dtype: int64

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

total_df[['도시', '구', '동']] = total_org['시군구'].str.extract(location_pattern)
total_df = total_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

### **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [ ]:
train_org['도로형태'].value_counts()

단일로 - 기타           19485
교차로 - 교차로안          9931
교차로 - 교차로부근         5746
기타 - 기타             1986
교차로 - 교차로횡단보도내      1474
단일로 - 지하차도(도로)내      309
주차장 - 주차장            236
단일로 - 교량             232
단일로 - 고가도로위          138
단일로 - 터널              64
미분류 - 미분류              8
Name: 도로형태, dtype: int64

In [ ]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

total_df[['도로형태1', '도로형태2']] = total_org['도로형태'].str.extract(road_pattern)
total_df = total_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

### Age(나이) 변수 전처리

In [ ]:
def age_transform(x):
  try:
    ret = int(x.split('세')[0])
  except:
    ret = np.NaN

  return ret

In [ ]:
train_df['가해운전자 연령'] = train_df['가해운전자 연령'].apply(lambda x:age_transform(x))
train_df['피해운전자 연령'] = train_df['피해운전자 연령'].apply(lambda x:age_transform(x))

total_df['가해운전자 연령'] = total_df['가해운전자 연령'].apply(lambda x:age_transform(x))
total_df['피해운전자 연령'] = total_df['피해운전자 연령'].apply(lambda x:age_transform(x))

### **외부데이터 - 전국 사고 데이터 전처리 및 이상치 제거**

In [ ]:
# train_df(대구 데이터)와 total_df(전체 도시 데이터)의 object형 column 확인

temp_obj_col = []
tot_obj_col = []
for col in train_df.columns:
    if train_df[col].dtype == object:
        temp_obj_col.append(col)
for col in total_df.columns:
    if total_df[col].dtype == object:
        tot_obj_col.append(col)

In [ ]:
# 대구 데이터에 없는 object형 변수 값을 갖고 있는 전체 도시 데이터의 인덱스를 파악
not_using_cols = ['ID', '도시', '구', '동']

idxs = []
for col in tot_obj_col:
    if col not in not_using_cols:
        temp_values = list(train_df[col].unique())
        total_values = list(total_df[col].unique())

        unique = []

        for tvalue in total_values:
            if tvalue not in temp_values:
                unique.append(tvalue)

        for u in unique:
            idx = total_df[total_df[col]==u].index.to_list()
            idxs = idxs + idx

idxs = list(set(idxs))

In [ ]:
total_df = total_df.drop(idxs, axis=0)
total_df.reset_index(inplace=True, drop=True)

In [ ]:
# 차대차, 차대사람의 데이터
not_car = total_df[total_df['사고유형'] != '차량단독'].copy()
not_car.reset_index(inplace=True, drop=True)

# 차량단독의 데이터
car = total_df[total_df['사고유형'] == '차량단독'].copy()
car.reset_index(inplace=True, drop=True)

In [ ]:
# 차량단독에서 ('노면상태', '가해운전자 연령', '도시', '구', '동') column에 대한 결측치 drop
car = car.dropna(subset=['노면상태', '가해운전자 연령', '도시', '구', '동'])
car.reset_index(inplace=True, drop=True)

# 차량단독의 경우 피해운전자 차종은 미분류나 결측치임. 따라서 없음으로 대치
car['피해운전자 차종'] = '없음'

# 차량단독이 아닌 경우에 대해서는 결측치 모두 drop
not_car = not_car.dropna()
not_car.reset_index(inplace=True, drop=True)

In [ ]:
# 차량단독과 차량단독이 아닌 경우를 합쳐서 전체 데이터 형성
country_df = pd.concat([car, not_car], axis=0)
country_df.reset_index(inplace=True, drop=True)

In [ ]:
# 이상치 제거(0.1%)
def set_limit(column):
    return np.quantile(column, 0.999)

country_copy = country_df.copy()

country_copy = country_copy[['기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '도로형태1', '도로형태2', 'ECLO']]
country_copy.reset_index(inplace=True, drop=True)

outlier_idxs = []
for col in country_copy.columns[:-1]:
    temp_df = pd.DataFrame(country_copy.groupby(col)['ECLO'].agg(set_limit)).reset_index()
    for j in range(len(temp_df)):
        s_idxs = country_df[(country_df[col] == temp_df.loc[j, col]) & (country_df['ECLO'] > temp_df.loc[j, 'ECLO'])].index.to_list()
        outlier_idxs = outlier_idxs + s_idxs
outliers = list(set(outlier_idxs))

print('outlier 수 : ', len(outliers))

country_df = country_df.drop(outliers, axis=0)
country_df.reset_index(inplace=True, drop=True)

print(len(country_df))

outlier 수 :  853
590664


### **파생 변수 생성 4 : 지역별 가해운전자 & 피해운전자 평균 연령 추출**

In [ ]:
age_mean = train_df[['도시','구','동','가해운전자 연령','피해운전자 연령']].groupby(['도시', '구','동']).mean()
age_mean.columns = ['가해운전자 평균연령','피해운전자 평균연령']

In [ ]:
country_mean = country_df[['도시','가해운전자 연령','피해운전자 연령']].groupby(['도시']).mean()
country_mean.columns = ['가해운전자 평균연령','피해운전자 평균연령']

In [ ]:
train_df = pd.merge(train_df, age_mean, how='left', on=['도시', '구','동'])
test_df = pd.merge(test_df, age_mean, how='left', on=['도시', '구','동'])

country_df = pd.merge(country_df, country_mean, how='left', on=['도시'])

In [ ]:
print(len(train_df.columns))
print(len(country_df.columns))

31
31


### **파생 변수 생성 5 : 지역별 가해운전자 & 피해운전자 평균 성별 추출**

In [ ]:
def sex_transform(x):
  if x=='남':
    return 1
  elif x=='여':
    return 0
  else:
    return np.NaN

In [ ]:
train_df['가해운전자 성별'] = train_df['가해운전자 성별'].apply(lambda x:sex_transform(x))
train_df['피해운전자 성별'] = train_df['피해운전자 성별'].apply(lambda x:sex_transform(x))
country_df['가해운전자 성별'] = country_df['가해운전자 성별'].apply(lambda x:sex_transform(x))
country_df['피해운전자 성별'] = country_df['피해운전자 성별'].apply(lambda x:sex_transform(x))

In [ ]:
sex_mean = train_df[['도시','구','동','가해운전자 성별','피해운전자 성별']].groupby(['도시', '구','동']).mean()
sex_mean.columns = ['가해운전자 평균성별','피해운전자 평균성별']

con_sex_mean = country_df[['도시','가해운전자 성별','피해운전자 성별']].groupby(['도시']).mean()
con_sex_mean.columns = ['가해운전자 평균성별','피해운전자 평균성별']

In [ ]:
train_df = pd.merge(train_df, sex_mean, how='left', on=['도시', '구','동'])
test_df = pd.merge(test_df, sex_mean, how='left', on=['도시', '구','동'])

country_df = pd.merge(country_df, con_sex_mean, how='left', on=['도시'])

### **파생 변수 생성 6 : 주말 변수 추가**

In [ ]:
def weekend(x):
  if x in ['토요일','일요일']:
    return 1
  else:
    return 0

train_df['주말'] = train_df['요일'].apply(lambda x:weekend(x))
test_df['주말'] = test_df['요일'].apply(lambda x:weekend(x))

country_df['주말'] = country_df['요일'].apply(lambda x:weekend(x))

### **전국데이터 + 대구데이터 Concat**

In [ ]:
print(np.unique(train_df.columns == country_df.columns))

[ True]


In [ ]:
train_df = pd.concat([train_df, country_df], axis=0)
train_df.reset_index(inplace=True, drop=True)
train_df.head(5)

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,도시,구,동,도로형태1,도로형태2,가해운전자 평균연령,피해운전자 평균연령,가해운전자 평균성별,피해운전자 평균성별,주말
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,0.0,51.0,...,대구광역시,중구,대신동,단일로,기타,52.714286,48.015504,0.753968,0.604651,0
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,보도통행중,기타,승용,1.0,39.0,...,대구광역시,달서구,감삼동,단일로,기타,47.176383,42.906767,0.744395,0.732331,0
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,1.0,70.0,...,대구광역시,수성구,두산동,단일로,기타,47.008547,43.270134,0.747009,0.756711,0
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,1.0,49.0,...,대구광역시,북구,복현동,단일로,기타,46.894089,42.943902,0.800493,0.714634,0
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,1.0,30.0,...,대구광역시,동구,신암동,단일로,기타,49.426415,47.400254,0.779874,0.738247,0


In [ ]:
len(train_df)

630273

### **파생 변수 생성 7 - 외부데이터(일출일몰시각 : 해 떴는지(1) 안떴는지(0))**

In [ ]:
def sun_transform(x, num):
    try:
        ret = int(x.split()[num])
    except:
        ret = np.NaN

    return ret


def day_transform(x, num):
    try:
        if ' 'in x.strip():
            ret = int(x.split()[num])
    except:
        ret = x

    return ret

sun_raw = pd.read_excel('./open/external_open/한국표준과학연구원_천문역법참조표준데이터_20161201.xlsx')
sun_raw = sun_raw.drop([0], axis=0)
sun_raw = sun_raw.drop(['일남중', 'Unnamed: 4', '일남중.1'], axis=1)

sun1 = sun_raw.iloc[:,:3] #상반기
sun2 = sun_raw.iloc[:,3:] #하반기
sun1.columns = ['일자', '일출', '일몰']
sun2.columns = sun1.columns
sun = pd.concat([sun1, sun2])
sun = sun.dropna(axis=0)



sun['월'] = sun['일자'].apply(lambda x:sun_transform(x, 0))
sun['월'] = sun['월'].fillna(method='ffill')
sun['일'] = sun['일자'].apply(lambda x:day_transform(x, 1))
sun['일출 시'] = sun['일출'].apply(lambda x:sun_transform(x, 0))
sun['일출 분'] = sun['일출'].apply(lambda x:sun_transform(x, 1))
sun['일몰 시'] = sun['일몰'].apply(lambda x:sun_transform(x, 0))
sun['일몰 분'] = sun['일몰'].apply(lambda x:sun_transform(x, 1))
sun['일출'] = sun['일출 시'] + sun['일출 분']/60
sun['일몰'] = sun['일몰 시'] + sun['일몰 분']/60
sun = sun[['월','일','일출','일몰']]

In [ ]:
tqdm.pandas()
def sun_func(x):
  sun_rise, sun_down = sun.loc[(sun['월']==x['월'])&(sun['일']==x['일']),['일출','일몰']].values[0]
  time = x['시간']
  if (sun_rise<time) and (time<sun_down):
    return 1
  else:
    return 0

train_df['sun'] = train_df[['월', '일', '시간']].progress_apply(lambda x:sun_func(x),axis=1)
test_df['sun'] = test_df[['월', '일', '시간']].progress_apply(lambda x:sun_func(x),axis=1)

100%|██████████| 10963/10963 [00:14<00:00, 738.85it/s]


### **파생 변수 생성 8 : 지역별 위험도 변수 생성**

#### ride dangerous(가해운전자 차종별 위험도를 측정 후 구 별로 가해운전자 차종 사고 발생 비율을 반영하여 가중치 생성)

In [ ]:
ride_dangerous = train_df[['가해운전자 차종','ECLO']].groupby('가해운전자 차종').mean()
ride_dangerous.columns = ['ride_dangerous']

In [ ]:
ride_count = train_df[['도시','구','가해운전자 차종']]
ride_count['cnt'] = 1
ride_count = ride_count.groupby(['도시','구','가해운전자 차종']).count()
ride_count.reset_index(inplace=True)

In [ ]:
temp_1 = pd.merge(ride_count,ride_dangerous,how='left',on=['가해운전자 차종'])
temp_1['multiply'] = temp_1['cnt']*temp_1['ride_dangerous']
temp_1

,도시,구,가해운전자 차종,cnt,ride_dangerous,multiply
0,강원도,강릉시,개인형이동수단(PM),2,3.542149,7.084298
1,강원도,강릉시,건설기계,30,5.116494,153.494833
2,강원도,강릉시,기타불명,2,3.143249,6.286498
3,강원도,강릉시,농기계,2,5.398464,10.796928
4,강원도,강릉시,사륜오토바이(ATV),1,4.844367,4.844367
...,...,...,...,...,...,...
2496,충청북도,충주시,원동기,67,4.199314,281.354021
2497,충청북도,충주시,이륜,165,4.364282,720.106605
2498,충청북도,충주시,자전거,103,3.518251,362.379836
2499,충청북도,충주시,특수,35,5.192286,181.730022


In [ ]:
temp_1 = temp_1.groupby(['도시','구']).sum().reset_index().drop(['ride_dangerous'],axis=1)
temp_1

,도시,구,cnt,multiply
0,강원도,강릉시,2376,11840.765601
1,강원도,고성군,385,1922.612006
2,강원도,동해시,1012,5056.794667
3,강원도,삼척시,1019,5049.303108
4,강원도,속초시,1214,6030.944008
...,...,...,...,...
227,충청북도,제천시,2368,11738.418111
228,충청북도,증평군,580,2838.914262
229,충청북도,진천군,1459,7240.488571
230,충청북도,청주시,13299,65509.364670


In [ ]:
temp_1['ride_dangerous'] = temp_1['multiply']/temp_1['cnt']
temp_1.drop(['multiply','cnt'],axis=1,inplace=True)
temp_1

,도시,구,ride_dangerous
0,강원도,강릉시,4.983487
1,강원도,고성군,4.993797
2,강원도,동해시,4.996833
3,강원도,삼척시,4.955155
4,강원도,속초시,4.967829
...,...,...,...
227,충청북도,제천시,4.957102
228,충청북도,증평군,4.894680
229,충청북도,진천군,4.962638
230,충청북도,청주시,4.925887


#### accident case dangerous(사고유형별 위험도를 측정 후 구 별로 사고유형 발생 비율을 반영하여 가중치 생성)

In [ ]:
accident_case_dangerous = train_df[['사고유형','ECLO']].groupby('사고유형').mean()
accident_case_dangerous.columns = ['accident_case_dangerous']
accident_case_dangerous

,accident_case_dangerous
사고유형,
차대사람,4.061413
차대차,5.127841
차량단독,4.575411


In [ ]:
accident_case_count = train_df[['도시','구','사고유형']]
accident_case_count['cnt'] = 1
accident_case_count = accident_case_count.groupby(['도시','구','사고유형']).count()
accident_case_count.reset_index(inplace=True)
accident_case_count

,도시,구,사고유형,cnt
0,강원도,강릉시,차대사람,466
1,강원도,강릉시,차대차,1858
2,강원도,강릉시,차량단독,52
3,강원도,고성군,차대사람,50
4,강원도,고성군,차대차,322
...,...,...,...,...
690,충청북도,청주시,차대차,11090
691,충청북도,청주시,차량단독,429
692,충청북도,충주시,차대사람,425
693,충청북도,충주시,차대차,2982


In [ ]:
temp_2 = pd.merge(accident_case_count,accident_case_dangerous,how='left',on=['사고유형'])
temp_2['multiply'] = temp_2['cnt']*temp_2['accident_case_dangerous']

In [ ]:
temp_2 = temp_2.groupby(['도시','구']).sum().reset_index().drop(['accident_case_dangerous'],axis=1)
temp_2['accident_case_dangerous'] = temp_2['multiply']/temp_2['cnt']
temp_2.drop(['multiply','cnt'],axis=1,inplace=True)
temp_2

,도시,구,accident_case_dangerous
0,강원도,강릉시,4.906594
1,강원도,고성군,4.970690
2,강원도,동해시,4.897470
3,강원도,삼척시,4.892425
4,강원도,속초시,4.900463
...,...,...,...
227,충청북도,제천시,4.946429
228,충청북도,증평군,4.932014
229,충청북도,진천군,4.978092
230,충청북도,청주시,4.967285


#### merge dangerous

In [ ]:
train_df = pd.merge(train_df,temp_1,how='left',on=['도시','구'])
test_df = pd.merge(test_df,temp_1,how='left',on=['도시','구'])

train_df = pd.merge(train_df,temp_2,how='left',on=['도시','구'])
test_df = pd.merge(test_df,temp_2,how='left',on=['도시','구'])

### **파생 변수 생성 9 : 외부데이터 - 택시 통행량**

In [ ]:
train_df2=train_df.copy()

In [ ]:
taxi = pd.read_csv('./open/external_open/국토교통부_택시운행정보관리시스템_전국 노드링크별 평균택시통행량(2021년).csv', encoding='CP949')
taxi = taxi.rename(columns={' 평균통행량 ':'평균통행량'})
taxi['도시']=taxi['지역'].str.split().str[0]

In [ ]:
lst0=train_df[train_df['도시']=='강원도']['구'].unique().tolist()
lst1=train_df[train_df['도시']=='경기도']['구'].unique().tolist()
lst2=train_df[train_df['도시']=='경상남도']['구'].unique().tolist()
lst3=train_df[train_df['도시']=='경상북도']['구'].unique().tolist()
lst4=train_df[train_df['도시']=='광주광역시']['구'].unique().tolist()
lst5=train_df[train_df['도시']=='대구광역시']['구'].unique().tolist()
lst6=train_df[train_df['도시']=='대전광역시']['구'].unique().tolist()
lst7=train_df[train_df['도시']=='부산광역시']['구'].unique().tolist()
lst8=train_df[train_df['도시']=='서울특별시']['구'].unique().tolist()
lst9=train_df[train_df['도시']=='세종특별자치시']['구'].unique().tolist()
lst10=train_df[train_df['도시']=='울산광역시']['구'].unique().tolist()
lst11=train_df[train_df['도시']=='인천광역시']['구'].unique().tolist()
lst12=train_df[train_df['도시']=='전라남도']['구'].unique().tolist()
lst13=train_df[train_df['도시']=='전라북도']['구'].unique().tolist()
lst14=train_df[train_df['도시']=='제주특별자치도']['구'].unique().tolist()
lst15=train_df[train_df['도시']=='충청남도']['구'].unique().tolist()
lst16=train_df[train_df['도시']=='충청북도']['구'].unique().tolist()

In [ ]:
train_df.drop(train_df[train_df['도시']=='미분류'].index,axis=0,inplace=True)

In [ ]:
temp={'강원도':lst0,'경기도':lst1,'경상남도':lst2,'경상북도':lst3,'광주광역시':lst4,'대구광역시':lst5,'대전광역시':lst6,
     '부산광역시':lst7,'서울특별시':lst8,'세종특별자치시':lst9,'울산광역시':lst10,'인천광역시':lst11,'전라남도':lst12,'전라북도':lst13,
     '제주특별자치도':lst14,'충청남도':lst15,'충청북도':lst16}

In [ ]:
def get_gu(x):
    for gu in temp[x.split()[0]]:
        if gu in str(x): return gu
    return None

In [ ]:
taxi['구']=taxi['지역'].apply(lambda x: get_gu(x))
taxi_df=taxi[['도시','구','평균통행량']].groupby(['도시','구']).agg(['sum','mean']).reset_index(drop=False)
taxi_df.columns=['도시','구','총통행량','평균통행량']
train_df=pd.merge(train_df,taxi_df,how='left',on=['도시','구'])

chong_dct=taxi_df.groupby('도시')['총통행량'].mean().to_dict() # 도시별 총통행량 평균
mean_dct=taxi_df.groupby('도시')['평균통행량'].mean().to_dict() # 도시별 평균통행량 평균

In [ ]:
train_df['총통행량'].fillna(train_df['도시'].map(chong_dct),inplace=True)
train_df['평균통행량'].fillna(train_df['도시'].map(mean_dct),inplace=True)

In [ ]:
test_df=pd.merge(test_df,taxi_df,how='left',on=['도시','구'])
test_df['총통행량'].fillna(test_df['도시'].map(chong_dct),inplace=True) # 도시별 총통행량 평균으로 fillna
test_df['평균통행량'].fillna(test_df['도시'].map(mean_dct),inplace=True) # 도시별 평균통행량 평균으로 fillna

In [ ]:
test_df[['총통행량','평균통행량']].isnull().sum(),train_df[['총통행량','평균통행량']].isnull().sum()

(총통행량     0
 평균통행량    0
 dtype: int64,
 총통행량     0
 평균통행량    0
 dtype: int64)

### **파생 변수 생성 10 : 외부데이터 - 택시 속도**

In [ ]:
#['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대전광역시', '미분류', '부산광역시', '서울특별시','세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']

In [ ]:
taxi = pd.read_csv('./open/external_open/국토교통부_택시운행정보관리시스템_전국 노드링크별 평균택시속도(2021년).csv', encoding='CP949')
taxi['도시']=taxi['지역'].str.split().str[0]
taxi['구']=taxi['지역'].apply(lambda x: get_gu(x))

In [ ]:
taxi_df = taxi[['도시','구','평균속도']].groupby(['도시','구']).mean().reset_index(drop=False)
speed_dct=taxi_df.groupby('도시')['평균속도'].mean().to_dict() # 도시별 평균속도 평균

In [ ]:
train_df=pd.merge(train_df,taxi_df,how='left',on=['도시','구'])
train_df['평균속도'].fillna(train_df['도시'].map(speed_dct),inplace=True) # 도시별 평균속도 평균으로 fillna

test_df=pd.merge(test_df,taxi_df,how='left',on=['도시','구'])
test_df['평균속도'].fillna(test_df['도시'].map(speed_dct),inplace=True) # 도시별 평균속도 평균으로 fillna

In [ ]:
test_df['평균속도'].isnull().sum(),train_df['평균속도'].isnull().sum()

(0, 0)

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

In [ ]:
display(train_df.head())
display(test_df.head())

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,피해운전자 평균연령,가해운전자 평균성별,피해운전자 평균성별,주말,sun,ride_dangerous,accident_case_dangerous,총통행량,평균통행량,평균속도
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,0.0,51.0,...,48.015504,0.753968,0.604651,0,0,4.814223,4.848318,459882.3,976.395541,22.072399
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,보도통행중,기타,승용,1.0,39.0,...,42.906767,0.744395,0.732331,0,0,4.864461,4.928028,720256.2,253.165624,23.630193
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,1.0,70.0,...,43.270134,0.747009,0.756711,0,0,4.871698,4.939717,536390.7,260.383835,25.842282
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,1.0,49.0,...,42.943902,0.800493,0.714634,0,0,4.822786,4.944707,647304.7,211.192398,24.690669
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,1.0,30.0,...,47.400254,0.779874,0.738247,0,0,4.848040,4.934366,577848.8,232.628341,26.543156


,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,...,피해운전자 평균연령,가해운전자 평균성별,피해운전자 평균성별,주말,sun,ride_dangerous,accident_case_dangerous,총통행량,평균통행량,평균속도
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,...,44.303951,0.728097,0.677812,1,0,4.871698,4.939717,536390.7,260.383835,25.842282
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,...,46.119247,0.685950,0.658996,1,0,4.871698,4.939717,536390.7,260.383835,25.842282
2,ACCIDENT_39611,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,...,45.321678,0.818182,0.664336,1,0,4.871698,4.939717,536390.7,260.383835,25.842282
3,ACCIDENT_39612,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,...,43.263014,0.671271,0.654795,1,0,4.871698,4.939717,536390.7,260.383835,25.842282
4,ACCIDENT_39613,토요일,맑음,건조,차대차,2022,1,1,6,대구광역시,...,42.906767,0.744395,0.732331,1,0,4.864461,4.928028,720256.2,253.165624,23.630193


## **train, test 데이터의 독립변수, 종속 변수 분리**

모델 추론 시점에서 획득할 수 있는 데이터는 학습 시점에서 활용할 수 있는 데이터와 다를 수 있습니다  
그렇기에 본 baseline에서는 test 시점에서만 활용할 수 있는 데이터들(test의 column들)만 사용하여 train 데이터를 구성했습니다

train_df 중에서 test_x가 가진 column들만 추출하여 train_x(독립변수)를 구성하고, train_df 중에서 예측 목표인 EPDO를 추출하여 train_y(종속변수)를 구성합니다

In [ ]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
train_x.drop(['동'],axis=1,inplace=True)
test_x.drop(['동'],axis=1,inplace=True)

In [ ]:
train_x.columns

Index(['요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '도로형태1',
       '도로형태2', '가해운전자 평균연령', '피해운전자 평균연령', '가해운전자 평균성별', '피해운전자 평균성별', '주말',
       'sun', 'ride_dangerous', 'accident_case_dangerous', '총통행량', '평균통행량',
       '평균속도'],
      dtype='object')

In [ ]:
train_x.isnull().sum()

요일                         0
기상상태                       0
노면상태                       0
사고유형                       0
연                          0
월                          0
일                          0
시간                         0
도시                         0
구                          0
도로형태1                      0
도로형태2                      0
가해운전자 평균연령                 0
피해운전자 평균연령                 0
가해운전자 평균성별                 0
피해운전자 평균성별                 0
주말                         0
sun                        0
ride_dangerous             0
accident_case_dangerous    0
총통행량                       0
평균통행량                      0
평균속도                       0
dtype: int64

## **범주형(Categorical) 변수, 수치형 변수로 변환하기**

모델 학습을 위해 train_x의 문자열 타입의 컬럼들을 추출하고, LabelEncoder를 활용하여 이 컬럼들을 모두 수치형 변수로 변환해 보겠습니다

In [ ]:
one_hot_features=['기상상태','노면상태','사고유형','도로형태1','도로형태2']

train_oh=pd.get_dummies(train_x[one_hot_features])
test_oh=pd.get_dummies(test_x[one_hot_features])

# # train_x=pd.get_dummies(train_x)
# # test_x=pd.get_dummies(test_x)

for i in train_oh.columns:
    if i not in test_oh.columns:
        test_oh[i]=0

test_oh=test_oh[train_oh.columns]
display(len(test_oh.columns),len(train_oh.columns))

train_x.drop(one_hot_features,axis=1,inplace=True)
test_x.drop(one_hot_features,axis=1,inplace=True)

train_x=pd.concat([train_x,train_oh],axis=1)
test_x=pd.concat([test_x,test_oh],axis=1)

30

30

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    print(i)
    le = LabelEncoder()
    le=le.fit(train_x[i])
    train_x[i]=le.transform(train_x[i])

    for case in np.unique(test_x[i]):
        if case not in le.classes_:
            print('test case is not in classes')
            le.classes_ = np.append(le.classes_, case)
    test_x[i]=le.transform(test_x[i])

display(train_x.head())
display(test_x.head())

['요일', '도시', '구']

요일
도시
구


,요일,연,월,일,시간,도시,구,가해운전자 평균연령,피해운전자 평균연령,가해운전자 평균성별,...,도로형태2_고가도로위,도로형태2_교량,도로형태2_교차로부근,도로형태2_교차로안,도로형태2_교차로횡단보도내,도로형태2_기타,도로형태2_미분류,도로형태2_주차장,도로형태2_지하차도(도로)내,도로형태2_터널
0,6,2019,1,1,0,5,168,52.714286,48.015504,0.753968,...,0,0,0,0,0,1,0,0,0,0
1,6,2019,1,1,0,5,53,47.176383,42.906767,0.744395,...,0,0,0,0,0,1,0,0,0,0
2,6,2019,1,1,1,5,101,47.008547,43.270134,0.747009,...,0,0,0,0,0,1,0,0,0,0
3,6,2019,1,1,2,5,81,46.894089,42.943902,0.800493,...,0,0,0,0,0,1,0,0,0,0
4,6,2019,1,1,4,5,59,49.426415,47.400254,0.779874,...,0,0,0,0,0,1,0,0,0,0


,요일,연,월,일,시간,도시,구,가해운전자 평균연령,피해운전자 평균연령,가해운전자 평균성별,...,도로형태2_고가도로위,도로형태2_교량,도로형태2_교차로부근,도로형태2_교차로안,도로형태2_교차로횡단보도내,도로형태2_기타,도로형태2_미분류,도로형태2_주차장,도로형태2_지하차도(도로)내,도로형태2_터널
0,5,2022,1,1,1,5,101,46.719033,44.303951,0.728097,...,0,0,0,1,0,0,0,0,0,0
1,5,2022,1,1,1,5,101,47.977273,46.119247,0.685950,...,0,0,0,0,0,1,0,0,0,0
2,5,2022,1,1,4,5,101,46.923077,45.321678,0.818182,...,0,0,0,1,0,0,0,0,0,0
3,5,2022,1,1,4,5,101,47.569061,43.263014,0.671271,...,0,0,0,0,0,1,0,0,0,0
4,5,2022,1,1,6,5,53,47.176383,42.906767,0.744395,...,0,0,0,1,0,0,0,0,0,0


## **Model Train & Prediction**

전체 학습 데이터에 대하여 학습후 최종 예측을 합니다. 여기서는 DecisionTreeRegressor 모델로 학습하고 예측하는 예시 코드입니다.

In [ ]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.4 MB/s eta 0:00:00


In [ ]:
train_x.columns

Index(['요일', '연', '월', '일', '시간', '도시', '구', '가해운전자 평균연령', '피해운전자 평균연령',
       '가해운전자 평균성별', '피해운전자 평균성별', '주말', 'sun', 'ride_dangerous',
       'accident_case_dangerous', '총통행량', '평균통행량', '평균속도', '기상상태_기타', '기상상태_눈',
       '기상상태_맑음', '기상상태_비', '기상상태_안개', '기상상태_흐림', '노면상태_건조', '노면상태_기타',
       '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수', '사고유형_차대사람',
       '사고유형_차대차', '사고유형_차량단독', '도로형태1_교차로', '도로형태1_기타', '도로형태1_단일로',
       '도로형태1_미분류', '도로형태1_주차장', '도로형태2_고가도로위', '도로형태2_교량', '도로형태2_교차로부근',
       '도로형태2_교차로안', '도로형태2_교차로횡단보도내', '도로형태2_기타', '도로형태2_미분류', '도로형태2_주차장',
       '도로형태2_지하차도(도로)내', '도로형태2_터널'],
      dtype='object')

In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, early_stopping
from catboost import CatBoostRegressor
import optuna

from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import train_test_split

### model optuna definitions

In [ ]:
def xgb_modeling(X_train, y_train, X_valid, y_valid):
  def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'seed':42,
        'max_depth': trial.suggest_int('max_depth', 3, 15), # Extremely prone to overfitting!
        'n_estimators': trial.suggest_int('n_estimators', 300, 3000, 200), # Extremely prone to overfitting!
        'eta': trial.suggest_float('eta', 0.007, 0.013), # Most important parameter.
        'subsample': trial.suggest_discrete_uniform('subsample', 0.3, 1, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.4, 0.9, 0.1),
        'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.4, 0.9, 0.1),
    }

    model = XGBRegressor(**params, random_state=42, n_jobs=-1, objective='reg:squaredlogerror')
    bst_xgb = model.fit(X_train,y_train, eval_set = [(X_valid,y_valid)], eval_metric='rmsle', early_stopping_rounds=100,verbose=False)

    preds = bst_xgb.predict(X_valid)
    if (preds<0).sum()>0:
      print('negative')
      preds = np.where(preds>0,preds,0)
    loss = msle(y_valid,preds)

    return np.sqrt(loss)

  study_xgb = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler(seed=100))
  study_xgb.optimize(objective,n_trials=30,show_progress_bar=True)

  xgb_reg = XGBRegressor(**study_xgb.best_params, random_state=42, n_jobs=-1, objective='reg:squaredlogerror')
  xgb_reg.fit(X_train,y_train,eval_set = [(X_valid,y_valid)], eval_metric='rmsle', early_stopping_rounds=100,verbose=False)

  return xgb_reg,study_xgb

In [ ]:
def lgbm_modeling(X_train, y_train, X_valid, y_valid):
  def objective(trial):
    param = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = LGBMRegressor(**param, random_state=42, n_jobs=-1)
    bst_lgbm = model.fit(X_train,y_train, eval_set = [(X_valid,y_valid)], eval_metric='rmse',callbacks=[early_stopping(stopping_rounds=100)])

    preds = bst_lgbm.predict(X_valid)
    if (preds<0).sum()>0:
      print('negative')
      preds = np.where(preds>0,preds,0)
    loss = msle(y_valid,preds)

    return np.sqrt(loss)

  study_lgbm = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler(seed=100))
  study_lgbm.optimize(objective,n_trials=30,show_progress_bar=True)

  lgbm_reg = LGBMRegressor(**study_lgbm.best_params, random_state=42, n_jobs=-1)
  lgbm_reg.fit(X_train,y_train,eval_set = [(X_valid,y_valid)], eval_metric='rmse', callbacks=[early_stopping(stopping_rounds=100)])

  return lgbm_reg,study_lgbm

In [ ]:
def cat_modeling(X_train, y_train, X_valid, y_valid):
  def objective(trial):
    param = {
        'iterations':trial.suggest_int("iterations", 1000, 20000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }


    model = CatBoostRegressor(**param, random_state=42)
    #task_type="GPU",devices='0:1'
    bst_cat = model.fit(X_train,y_train, eval_set = [(X_valid,y_valid)], early_stopping_rounds=100,verbose=False)

    preds = bst_cat.predict(X_valid)
    if (preds<0).sum()>0:
      print('negative')
      preds = np.where(preds>0,preds,0)
    loss = msle(y_valid,preds)

    return np.sqrt(loss)

  study_cat = optuna.create_study(direction='minimize',sampler=optuna.samplers.TPESampler(seed=100))
  study_cat.optimize(objective,n_trials=30,show_progress_bar=True)

  cat_reg = CatBoostRegressor(**study_cat.best_params, random_state=42)
  cat_reg.fit(X_train,y_train,eval_set = [(X_valid,y_valid)], early_stopping_rounds=100,verbose=False)

  return cat_reg,study_cat

### xgb, cat, lgbm ensemble

In [ ]:
X_train,X_valid,y_train,y_valid = train_test_split(train_x,train_y,random_state=42,shuffle=True,test_size=0.2)

In [ ]:
# target 변수 왜도 첨도 조정 위해 log 처리

temp_train_y = np.log1p(y_train)
temp_valid_y = np.log1p(y_valid)

In [ ]:
cat,study_cat = cat_modeling(X_train,temp_train_y,X_valid,temp_valid_y)

In [ ]:
xgb,study_xgb = xgb_modeling(X_train,temp_train_y,X_valid,temp_valid_y)

In [ ]:
lgbm,study_lgbm = lgbm_modeling(X_train,temp_train_y,X_valid,temp_valid_y)

In [ ]:
cat_prediction = cat.predict(test_x)
xgb_prediction = xgb.predict(test_x)
lgbm_prediction = lgbm.predict(test_x)

In [ ]:
cat_prediction = np.expm1(cat_prediction)
xgb_prediction = np.expm1(xgb_prediction)
lgbm_prediction = np.expm1(lgbm_prediction)

In [ ]:
#xgb valid loss 작지만 제출 경험상 과적합으로 score 낮음

prediction = xgb_prediction * 0.1 + cat_prediction * 0.4 + lgbm_prediction * 0.5
submit = sample_submission.copy()
submit['ECLO'] = prediction

In [ ]:
submit.to_csv('final_submision.csv', index=False)

In [ ]:
# 모델 가중치 저장

import pickle
with open('final_xgb.pkl', 'wb') as f:
    pickle.dump(xgb, f)

with open('final_cat.pkl', 'wb') as f:
    pickle.dump(cat, f)

with open('final_lgbm.pkl', 'wb') as f:
    pickle.dump(lgbm, f)